# Carrot Catcher
#### This program takes footage from a camera, filters orange objects from each frame, feeds these filtered frames to the carrot identifying model, and then saves the model output to a MySQL database.

Throughout this assessment, I made a few assumptions, since we were supposed to assume that a model was already given to us. I will give explanations for these assumptions along with my thought process for the functions I implemented.

---
## Libraries
---

In [ ]:
import cv2
import tensorflow as tf
import mysql

---
## Set Up Camera and Model
---

#### Video Capture
Camera footage is collected live by using OpenCV's VideoCapture method. The default input, 0, just gets the default camera from your computer. If we were actually using this in a grocery store, then we would have to give the program the grocery store's camera path.

#### Model Set Up
The loading of ML models usually depends on a loading method from the library that was used to make the model. For this assessment, let's just assume that the model is a TensorFlow model. 

In [ ]:
# Video capture
cap = cv2.VideoCapture(0)
                                                                                                                              
# Load ML model
modelPath = "Insert Model Path Here"
model = tf.keras.models.load_model(modelPath) 

---
## Set Up Data Base
---

#### Database Decisions
As you can see, this isn't a real database. If we were to implement a real database, then we would obviously have to give real parameters. For our implmementation, I used MySQL to record carrot IDs from the model output. For a more sophisticated implementation, adding more output data would be ideal. For example, it might be a good idea to include details such as size of carrot, date of recording, and carrot location in the frame. Additionally, it could also be better to use Postegres instead of MySQL, so that we could use array types for location/coordinate data.

In [ ]:
# Load MySQL DB
db = mysql.connector.connect(
    user="user",
    password="password",
    host="host IP",
    database="database name"
)

cursor = db.cursor()

# Prepare DB insertions
query = ("INSERT IGNORE INTO carrots (carrot_id) " # IGNORE catches duplicates
        "VALUES (%s)")

def insertCarrot(carrotID):
    cursor.execute(query, carrotID)

---
## Image Cleaning
---
#### Why Filter The Image?
Some models can simply be given unfiltered images and categorize the objects in the frame. I initially went with this assumption, but I felt like I wasn't really showing my knowledge on categorization by only feeding a raw image to the model. Therefore, I decided to filter the orange colours from the image as a way to somewhat isolate the carrots from the rest of the image. 

#### Notes on Sources
I followed the general process for colour masking using [this video](https://www.youtube.com/watch?v=aFNDh5k3SjU). This involves taking the HSV (Hue, Saturation, and Value) measurements from the image and isolating the values that are in the HSV range of orange. I determined the range by following [this Stack Overflow post](https://stackoverflow.com/questions/10948589/choosing-the-correct-upper-and-lower-hsv-boundaries-for-color-detection-withcv). To be honest, colour isolation is pretty finicky, so my implementation is definitely not optimal. It would be ideal to run an algorithm to determine values based on each picture, rather than a blanket range for all oranges.

In [ ]:
# Create a mask for orange using the HSV values of the image
def filterOrange(img):
    hsvImg = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lowerLim = (12, 100, 100)
    upperLim = (25, 255, 255)
    mask = cv2.inRange(hsvImg, lowerLim, upperLim)
    newImg = cv2.bitwise_and(img, img ,mask= mask)
    return newImg

---
## Image Detection
---

### Output Assumptions
Models can output any kind of data. For my implementation, I just assumed that the model returned an array of IDs for each carrot found in the image. 

### General Program Layout
This kind of while loop is pretty common in OpenCV projects that analyze video data. As you can see, we check to see if the camera/video is working, clean the data, then put that image data into a model. Afterwards, we record the output and display the image in some way. For the purposes of this assessment, it wasn't necessary to display the image of the camera or to include a way to close the program, but I still included them, since they would be useful in a real implementation.

In [ ]:
while True:
    # Checks if camera/video is working
    success, img = cap.read()
    if not success:
        break

    if not img:
        print("No image could be found from your camera")
        break

    # Feed image to the model
    cleanedImg = filterOrange(img)
    results = model(cleanedImg) # Returns an id for each unique carrot found

    # Add each carrot to the DB
    for carrot in results:
        insertCarrot(carrotID=carrot)

    # Show camera footage
    cv2.imshow("Image", img)
     
    # Press ESC to close program
    if cv2.waitKey(1) == 27:
        break

---
## Final Thoughts
---
This little assessment was kind of fun. It reminded me of a different OpenCV project that I did last year. That one involved identifying the shape of guitar chords using a TensorFlow model. You can check out that project on [my repo here](https://github.com/benkyli/Guitar-Chord-Identifier).